# Create and run local RAG pipeline from scratch

In [1]:
import os
import requests

In [2]:


# get pdf path
pdf_path = "pdf_path"

# if path is not exists then download
if not os.path.exists(pdf_path):
  print(f"[INFO] File does not exists, downloading...")

  # enter the prefered url of your choice
  url = "url_path"

  # The local filename to save the downloaded file
  filename = pdf_path

  # send the GET requests to the url
  response = requests.get(url)

  # Check if response status code
  if response.status_code == 200:
    # open the file and save it
    with open(filename,"wb") as file:
      file.write(response.content)
    print(f"[INFO] The file has been downloaded and saved as {filename}")
  else:
    print(f"[INFO] Failed to download the file, status code: {response.status_code}")
else:
  print(f"File exists: {pdf_path}")



[INFO] File does not exists, downloading...
[INFO] The file has been downloaded and saved as human_nutrition.pdf


In [3]:
# to open pdf we use PyMuPDF
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 62.9 MB/s eta 0:00:00:00:0100:01


In [4]:
import fitz
from tqdm.auto import tqdm  # this is for progress bar

In [5]:
def text_formatter(text: str) -> str:
  """
  Performs minor formatting on text.
  """
  cleaned_text = text.replace("\n"," ").strip()

  # potentially more text formatting function go here
  return cleaned_text


def open_and_read_pdf(pdf_path: str) -> list[dict]:
  doc = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text = text)
    pages_and_texts.append({"page_number": page_number - 41,
                            "page_char_count": len(text),
                            "page_word_count": len(text.split(" ")),
                            "page_sentence_count_raw": len(text.split(sep = ". ")),
                            "page_token_count": len(text)/4,
                            "text": text})
  return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [6]:
import random
random.sample(pages_and_texts, k=3)

[{'page_number': 1072,
  'page_char_count': 1234,
  'page_word_count': 199,
  'page_sentence_count_raw': 9,
  'page_token_count': 308.5,
  'text': 'The Health At Every Size® Principles are:  1. Weight Inclusivity: Accepting and respecting the diversity of  body shapes and sizes  2. Health Enhancement: Recognizing that health and well-being  are multi-dimensional and that they include physical, social,  spiritual, occupational, emotional, and intellectual aspects  3. Respectful Care: Promoting all aspects of health and well- being for people of all sizes  4. Eating for Well-being: Promoting eating in a manner which  balances individualized nutritional needs, hunger, satiety,  appetite, and pleasure  5. Life-Enhancing Movement: Promoting individually  appropriate, enjoyable, life-enhancing physical activity, rather  than exercise that is focused on a goal of weight loss  In the Hawaiian language, Lokahi means “unity, agreement, accord,  and harmony”.23\xa0 The concept of Lokahi can be us

In [7]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [8]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593
std,348.86387,560.441673,95.747365,6.187226,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500
50%,562.50000,1232.500000,215.000000,10.000000,308.125000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500
max,1166.00000,2308.000000,429.000000,32.000000,577.000000


# splitting pages into sentences

Two ways to do this:
1. We've done this by splitting on ". "
2. We can do this using NLP such as spacy, nltk

In [9]:
from spacy.lang.en import English
nlp = English()

In [10]:
# add sentencizer pipeline, which turns text into a sentences
nlp.add_pipe("sentencizer")

# create documents instance for an example
doc = nlp("This is a sentence. This another sentence. I like Elephants")
assert len(list(doc.sents))==3

list(doc.sents)

[This is a sentence., This another sentence., I like Elephants]

In [11]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)


  # making a sentence into a string which is always a good practice
  item["sentences"] = [str(sentence) for sentence in item["sentences"]]

  # count the sentences
  item["page_sentence_count_spacy"] = len(item["sentences"])



  0%|          | 0/1208 [00:00<?, ?it/s]

In [12]:
random.sample(pages_and_texts, k=1)

[{'page_number': 880,
  'page_char_count': 1769,
  'page_word_count': 279,
  'page_sentence_count_raw': 20,
  'page_token_count': 442.25,
  'text': 'aged children. Recent studies show that three million children  under age eighteen are allergic to at least one type of food.6  Some of the most common allergenic foods include peanuts, milk,  eggs, soy, wheat, and shellfish. An allergy occurs when a protein  in food triggers an immune response, which results in the release  of antibodies, histamine, and other defenders that attack foreign  bodies. Possible symptoms include itchy skin, hives, abdominal pain,  vomiting, diarrhea, and nausea. Symptoms usually develop within  minutes to hours after consuming a food allergen. Children can  outgrow a food allergy, especially allergies to wheat, milk, eggs, or  soy.  Anaphylaxis is a life-threatening reaction that results in difficulty  breathing, swelling in the mouth and throat, decreased blood  pressure, shock, or even death. Milk, eggs, whea

In [13]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.59,198.89,9.97,287.15,10.32
std,348.86,560.44,95.75,6.19,140.11,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.75,134.00,4.00,190.69,5.00
50%,562.50,1232.50,215.00,10.00,308.12,10.00
75%,864.25,1605.25,271.25,14.00,401.31,15.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00


### chunking our sentences together

The concept of splitting larger pieces of text into smaller one is often referred to as text spltting or chunking.

In [14]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# create a function to split list of texts recursively into chunk size
# e.g. -> [20] -> [10,10]
def split_size(input_list: list[str],
               slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
  return [input_list[i:i + slice_size] for i in range(0, len(input_list) + 1, slice_size)]

test_list = list(range(25))
split_size(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [15]:
# loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_size(input_list = item["sentences"],
                                       slice_size = num_sentence_chunk_size)
  item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [16]:
random.sample(pages_and_texts, k = 1)

[{'page_number': 0,
  'page_char_count': 762,
  'page_word_count': 133,
  'page_sentence_count_raw': 4,
  'page_token_count': 190.5,
  'text': 'Special Thanks to  Bill Chismar – University of Hawai’i at Mānoa, Dean of Outreach  College  The Children’s Healthy Living Summer Institute – University of  Hawai’i at Mānoa  Open Educational Resources  This text is provided to you as an Open Educational Resource (OER)  which you access online. It is designed to give you a comprehensive  introduction to human nutrition at no or very nominal cost. \xa0It  contains both written and graphic text material, intra-text links to  other internal material which may aid in understanding topics and  concepts, intra-text links to the appendices and glossary for tables  and definitions of words, and extra-text links to videos and web  material that clarifies and augments topics and concepts.  xlii  |  Acknowledgements',
  'sentences': ['Special Thanks to  Bill Chismar – University of Hawai’i at Mānoa, Dean 

In [17]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.59,198.89,9.97,287.15,10.32,1.62
std,348.86,560.44,95.75,6.19,140.11,6.30,0.63
min,-41.00,0.00,1.00,1.00,0.00,0.00,1.00
25%,260.75,762.75,134.00,4.00,190.69,5.00,1.00
50%,562.50,1232.50,215.00,10.00,308.12,10.00,2.00
75%,864.25,1605.25,271.25,14.00,401.31,15.00,2.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00,3.00


#### splitting each chunks into its own item



*   We'd like to embed each chunk of sentences into its own numerical representation
*   That'll give us good level of granularity.
Meaning, we can dive specifically into the text sample that was used in our model

In [18]:
import re

# split each chunk in its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
  for sentence_chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["page_number"] = item["page_number"]

    # join the sentences together into a paragraph like structure
    joined_sentence_chunk = "".join(sentence_chunk).replace("  "," ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # it will go from ".A" -> ". A"

    chunk_dict["sentence_chunk"] = joined_sentence_chunk

    # get some stats of the chunks
    chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"] = len([joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence_chunk)/4 # 1 token ~ 4 char

    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1208 [00:00<?, ?it/s]

1956

In [19]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 1037,
  'sentence_chunk': 'refrigerator and discard leftover marinade after use because it contains raw juices. Always use clean cutting boards, which should be washed with soap and warm water by hand or in a dishwasher after each use. Another way to sanitize cutting boards is to rinse them with a solution of 5 milliliters (1 teaspoon) chlorine bleach to about 1 liter (1 quart) of water. If possible, use separate cutting boards for fresh produce and for raw meat. Also, wash the top before opening canned foods to prevent dirt from coming into contact with food. Cooking Food Cooked food is safe to eat only after it has been heated to an internal temperature that is high enough to kill bacteria. You cannot judge the state of “cooked” by color and texture alone. Instead, use a food thermometer to be sure. The appropriate minimum cooking temperature varies depending on the type of food. Seafood should be cooked to an internal temperature of 145°F, beef, lamb, and pork to 16

In [20]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1956.00,1956.00,1956.0,1956.00
mean,586.88,692.38,1.0,173.09
std,348.20,466.94,0.0,116.73
min,-41.00,0.00,1.0,0.00
25%,283.00,264.75,1.0,66.19
50%,592.50,700.00,1.0,175.00
75%,894.00,1097.00,1.0,274.25
max,1166.00,1831.00,1.0,457.75


max embedding model has were 384 and here it's 457 for max sample that means we are losing some information

### Creating a filter to get out small irrelevant information like heading

In [21]:
# show  random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(10).iterrows():
  print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 12.75 | Text: https:/ /www.fda.gov/food/ 1022 | Food Preservation
Chunk token count: 0.0 | Text: 
Chunk token count: 25.0 | Text: U. S. Food and Drug Administration. https:/ /www.fda.gov/food/ 1030 | The Effect of New Technologies
Chunk token count: 28.5 | Text: The World Bank claims that a million deaths occur every year from anemia and that the majority of those Iron | 663
Chunk token count: 0.0 | Text: 
Chunk token count: 17.0 | Text: Figure 2.15 Major Respiratory Structures The Respiratory System | 99
Chunk token count: 0.0 | Text: 
Chunk token count: 13.25 | Text: 1036 | Efforts on the Consumer Level: What You Can Do
Chunk token count: 16.25 | Text: Table 14.2  Micronutrient Levels during Puberty 886 | Adolescence
Chunk token count: 11.75 | Text: Accessed March 17, 2018. Sports Nutrition | 961


In [22]:
df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,-41,Human Nutrition: 2020 Edition,29,1,7.25
1,-40,,0,1,0.00
2,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,1,77.00
3,-38,Human Nutrition: 2020 Edition by University of...,210,1,52.50
4,-37,Contents Preface University of Hawai‘i at Māno...,766,1,191.50


In [23]:
# filter out dataframe for rows with under 30 tokens
pages_and_chunks_over_min_token_length = df[df["chunk_token_count"]> min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_length[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 1,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 1,
  'chunk_token_count': 52.5}]

In [24]:
random.sample(pages_and_chunks_over_min_token_length, k= 1)

[{'page_number': 892,
  'sentence_chunk': 'Micronutrients Micronutrient recommendations for adolescents are mostly the same as for adults, though children this age need more of certain minerals to promote bone growth (e.g., calcium and phosphorus, along with iron and zinc for girls). Again, vitamins and minerals should be obtained from food first, with supplementation for certain micronutrients only (such as iron). The most important micronutrients for adolescents are calcium, vitamin D, vitamin A, and iron. Adequate calcium and vitamin D are essential for building bone mass. The recommendation for calcium is 1,300 milligrams for both boys and girls. Low-fat milk and cheeses are excellent sources of calcium and help young people avoid saturated fat and cholesterol. It can also be helpful for adolescents to consume products fortified with calcium, such as breakfast cereals and orange juice. Iron supports the growth of muscle and lean body mass. Adolescent girls also need to ensure suffi

### Done with PreProcessing of Data

### Now with embedding
## Embedding with our Texts

In [25]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.5 MB/s eta 0:00:00a 0:00:01m


In [26]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu")

# create a  list of sentences
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sentence: The Sentences Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-2.07981709e-02  3.03164534e-02 -2.01217774e-02  6.86483979e-02
 -2.55256128e-02 -8.47692229e-03 -2.07097939e-04 -6.32377863e-02
  2.81605516e-02 -3.33353803e-02  3.02634630e-02  5.30720875e-02
 -5.03526218e-02  2.62287799e-02  3.33314128e-02 -4.51578274e-02
  3.63044292e-02 -1.37119228e-03 -1.20171597e-02  1.14946393e-02
  5.04510887e-02  4.70856652e-02  2.11913232e-02  5.14608175e-02
 -2.03746185e-02 -3.58889811e-02 -6.67826505e-04 -2.94393282e-02
  4.95859124e-02 -1.05639771e-02 -1.52012929e-02 -1.31755811e-03
  4.48197126e-02  1.56023502e-02  8.60379544e-07 -1.21389201e-03
 -2.37979032e-02 -9.09352035e-04  7.34487502e-03 -2.53930152e-03
  5.23370802e-02 -4.68044803e-02  1.66214425e-02  4.71579209e-02
 -4.15599830e-02  9.01890744e-04  3.60278338e-02  3.42214443e-02
  9.68227386e-02  5.94828725e-02 -1.64984725e-02 -3.51250023e-02
  5.92520554e-03 -7.07940664e-04 -2.4103

In [27]:
embeddings[0].shape

(768,)

In [28]:
embedding = embedding_model.encode("I love protein")
embedding

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([ 2.13659219e-02,  8.78888071e-02,  2.04071850e-02, -3.54401320e-02,
        5.37913442e-02,  3.16504911e-02, -1.38830096e-01,  2.24571750e-02,
       -1.05773406e-02, -1.51525186e-02, -1.04130618e-02, -2.74976734e-02,
        2.17646733e-02,  6.80073276e-02,  1.16549609e-02,  1.77874044e-02,
        9.04292148e-03,  3.35060284e-02,  2.05003526e-02,  2.17815395e-02,
       -2.65372824e-02,  5.94804948e-03, -1.91729970e-03, -2.30704378e-02,
        2.46178918e-03,  2.22445540e-02,  1.24289719e-02, -3.33139412e-02,
        9.57538746e-03,  4.62307408e-02, -7.32055083e-02,  6.89381547e-03,
        4.16604616e-03, -1.56981517e-02,  1.40538600e-06,  5.04040858e-03,
       -3.56189422e-02,  2.81684250e-02, -5.60051762e-02,  9.07522291e-02,
       -9.67106000e-02, -3.82632874e-02, -4.47417274e-02, -5.61124906e-02,
       -1.32234916e-02,  6.91277832e-02,  3.74381095e-02, -1.83117054e-02,
       -3.07612475e-02, -6.30257558e-03, -6.43640477e-03, -2.52940282e-02,
       -2.51006112e-02,  

In [ ]:
%%time

embedding_model.to("cpu")

# embed each chunk one by one
for item in tqdm(pages_and_chunks_over_min_token_length):
  item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/1680 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# %%time

# embedding_model.to("cuda")

# # embed the sentences
# for item in tqdm(pages_and_chunks_over_min_token_length):
#   item["embedding"] = embedding_model.encode(item["sentence_chunk"])

In [ ]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_length]
text_chunks[419]

In [ ]:
len(text_chunks)

In [33]:
%%time
# Embed all the chunks in batches

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=16,
                                               convert_to_tensor=True)
text_chunk_embeddings

CPU times: user 11min 45s, sys: 2min 34s, total: 14min 20s
Wall time: 7min 10s


tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]])

# saving embedding them so that we can import it later

In [34]:
# save embeddings to the file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_length)
embedding_save_path_df = "text_chunks_and_embeddings.csv"
text_chunks_and_embeddings_df.to_csv(embedding_save_path_df, index=False)


In [35]:
# importing the embedding that we saved
text_chunks_and_embedding_df_load = pd.read_csv(embedding_save_path_df)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,1,77.00,[ 6.74242452e-02 9.02281702e-02 -5.09551540e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,1,52.50,[ 5.52156270e-02 5.92138991e-02 -1.66167244e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,1,191.50,[ 2.79802065e-02 3.39813679e-02 -2.06426680e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,1,235.25,[ 6.82566687e-02 3.81274857e-02 -8.46854132e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,1,249.50,[ 3.30264494e-02 -8.49767216e-03 9.57159046e-...


# RAG
RAG goal: is to retrieve passage base on query and use those passages to augment an input to an LLM so it can generate an output

### Similarity Search

In [36]:
import random
import pandas as pd
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# import texts and embeddings df
text_chunks_and_embedding_df = pd.read_csv("/kaggle/working/text_chunks_and_embeddings.csv")

# Convert this data which is in string into numpy array
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert embeddings into torch.tensor
# embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
# embeddings.shape

embeddings = torch.tensor(np.stack(text_chunks_and_embedding_df["embedding"].tolist(), axis = 0), dtype=torch.float32).to(device)

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

text_chunks_and_embedding_df

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,1,77.00,"[0.0674242452, 0.0902281702, -0.0050955154, -0..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,1,52.50,"[0.055215627, 0.0592138991, -0.0166167244, -0...."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,1,191.50,"[0.0279802065, 0.0339813679, -0.020642668, 0.0..."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,1,235.25,"[0.0682566687, 0.0381274857, -0.00846854132, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,1,249.50,"[0.0330264494, -0.00849767216, 0.00957159046, ..."
...,...,...,...,...,...,...
1675,1164,Flashcard Images Note: Most images in the flas...,1305,1,326.25,"[0.0185622796, -0.0164277926, -0.0127045782, -..."
1676,1164,Hazard Analysis Critical Control Points reused...,375,1,93.75,"[0.0334720723, -0.0570441373, 0.0151489237, -0..."
1677,1165,ShareAlike 11. Organs reused “Pancreas Organ A...,1286,1,321.50,"[0.0770515501, 0.0097855581, -0.0121817207, 0...."
1678,1165,Sucrose reused “Figure 03 02 05” by OpenStax B...,410,1,102.50,"[0.103045136, -0.0164702497, 0.00826846529, 0...."


In [37]:
embeddings.shape

torch.Size([1680, 768])

In [38]:
# Sentence Transformer model
from sentence_transformers import util, SentenceTransformer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device)

# embeddings = torch.tensor(embeddings).to(device)

Embedding model is ready!

In [39]:
# 1. Define the query
query = "protein"
print(f"Query: {query}")

# 2. Embed the query
query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")

# 3. Get Similarity score with dot or cos product if output is not normalized
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a= query_embedding, b= embeddings)[0]
end_time = timer()

print(f"[INFO] time taken to get score on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")

# 4. Top k results
top_k = 5
top_results_dot_product = torch.topk(dot_scores, k=top_k)
top_results_dot_product

Query: protein


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] time taken to get score on 1680 embeddings: 0.01281 seconds


torch.return_types.topk(
values=tensor([0.6415, 0.6255, 0.6180, 0.6115, 0.5915], device='cuda:0'),
indices=tensor([601, 584, 547,   9, 628], device='cuda:0'))

In [40]:
pages_and_chunks[601]

{'page_number': 405,
 'sentence_chunk': 'Image by Braden Collum on unsplash.co m / CC0 Proteins in a Nutshell UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM \xa0 Proteins are long chains of amino acids folded into precise structures that determine their functions, which are in the tens of thousands. They are the primary construction materials of the body serving as building blocks for bone, skin, hair, muscle, hormones, and antibodies. Without them we cannot breakdown or build macromolecules, grow, or heal from a wound. Too little protein impairs bodily functions and too much can lead to chronic disease. Eat proteins in moderation, at least 10 percent of the calories you take in and not more than 35 percent. Proteins are in a variety of foods. More complete sources are in animal-based foods, but choose those low in saturated fat and cholesterol. Some plant-based foods are also complete protein sources and don’t add much to your satur

# what if embeddings is from 1680 is 1000 times than this which is almost like a library which is also almost like 400 books of 700 pages

In [41]:
# larger_embeddings = torch.randn(1000*embeddings.shape[0],768).to(device)

# # perform dot product over 1680*1000 embeddings
# start_time = timer()
# dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
# end_time = timer()
# print(f"[INFO] time taken to get score on {len(larger_embeddings)} embeddings: {end_time - start_time:.5f} seconds")

Let's make our vector search result pretty

In [42]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)
    

In [43]:
print(f"Query: {query}")
print("Results:")

# loop through zipped together score and indices from torch.topk
for score, idx in zip(top_results_dot_product[0],top_results_dot_product[1]):
    print(f"Scores: {score:.4f}")
    print("Text:")
    print(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page Number: {pages_and_chunks[idx]['page_number']}")
    print("\n")


Query: protein
Results:
Scores: 0.6415
Text:
Image by Braden Collum on unsplash.co m / CC0 Proteins in a Nutshell UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM   Proteins are long chains of amino acids folded into precise structures that determine their functions, which are in the tens of thousands. They are the primary construction materials of the body serving as building blocks for bone, skin, hair, muscle, hormones, and antibodies. Without them we cannot breakdown or build macromolecules, grow, or heal from a wound. Too little protein impairs bodily functions and too much can lead to chronic disease. Eat proteins in moderation, at least 10 percent of the calories you take in and not more than 35 percent. Proteins are in a variety of foods. More complete sources are in animal-based foods, but choose those low in saturated fat and cholesterol. Some plant-based foods are also complete protein sources and don’t add much to your satu

In [ ]:
import fitz

# open pdf and load target 
pdf_path = "pdf_path"
doc = fitz.open(pdf_path)
page = doc.load_page(5 + 41)

# get the image of the page
img = page.get_pixmap(dpi=330)

#save the image is optional
# img.save("output_filename.png")
doc.close()

# convert the Pixmap to a numpy array
img_array = np.frombuffer(img.samples_mv,
                         dtype=np.uint8).reshape((img.h, img.w, img.n))

# Displaying it using matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(13,10))
plt.imshow(img_array)
plt.title(f"Query: {query} | Most Relevant Page: ")
plt.axis('off')
plt.show()


: 

### Similarity mesures: dot product and cosine similarity from scratch

In [45]:
import torch
def dot_product(vector1,vector2):
    return torch.dot(vector1, vector2)

def cosine_similarity(vector1,vector2):
    dot_product = torch.dot(vector1,vector2)
    
    # get the squareroot of each one
    norm_vector1 = torch.sqrt(torch.sum(vector1**2))
    norm_vector2 = torch.sqrt(torch.sum(vector2**2))
    
    return dot_product / (norm_vector1*norm_vector2)

In [46]:
# example vectors/tensors
vector1 = torch.tensor([1,2,3], dtype=torch.float32)
vector2 = torch.tensor([-1,-4,3], dtype=torch.float32)
vector3 = torch.tensor([3,2,3], dtype=torch.float32)
vector4 = torch.tensor([1,2,3], dtype=torch.float32)

# dot product between vectors
print(f"Dot product between vector1 and vector2: {dot_product(vector1, vector2)}")
print(f"Dot product between vector1 and vector3: {dot_product(vector1, vector3)}")
print(f"Dot product between vector1 and vector4: {dot_product(vector1, vector4)}")
print(f"Dot product between vector2 and vector3: {dot_product(vector2, vector3)}")

Dot product between vector1 and vector2: 0.0
Dot product between vector1 and vector3: 16.0
Dot product between vector1 and vector4: 14.0
Dot product between vector2 and vector3: -2.0


In [47]:
print(f"Cosine similarity between vector1 and vector2: {cosine_similarity(vector1, vector2)}")
print(f"Cosine similarity between vector1 and vector3: {cosine_similarity(vector1, vector3)}")
print(f"Cosine similarity between vector1 and vector4: {cosine_similarity(vector1, vector4)}")
print(f"Cosine similarity between vector2 and vector3: {cosine_similarity(vector2, vector3)}")

Cosine similarity between vector1 and vector2: 0.0
Cosine similarity between vector1 and vector3: 0.911684513092041
Cosine similarity between vector1 and vector4: 0.9999999403953552
Cosine similarity between vector2 and vector3: -0.08362419903278351


##### Functionizing semantic search pipeline

In [48]:
import time
from time import perf_counter as timer
def retrieve_relevant_resources(query: str, embeddings: torch.tensor, model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
  """
  Embed a query with model
  """
  # embed the query
  query_embedding = model.encode(query, convert_to_tensor=True)

  # get dot product scores on embeddings
  start_time = timer()
  dot_scores = util.dot_score(query_embedding, embeddings)[0]
  end_time = timer()
  if print_time:
    print(f"[INFO] Time taken to get score on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")

  scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
  return scores, indices

def top_result_and_scores(query: str,
                          embeddings: torch.tensor,
                          pages_and_chunks: list[dict]=pages_and_chunks,
                          n_resources_to_return: int=5):
  """
  Find relevant passages given a query and prints them out along withe tehir scores
  """
  scores, indices = retrieve_relevant_resources(query=query, embeddings=embeddings, n_resources_to_return=n_resources_to_return)

  # loop through zipped together score and indices from torch.topk
  for score, idx in zip(scores,indices):
    print(f"Scores: {score:.4f}")
    print("Text:")
    print(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page Number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

In [49]:
query= "foods high in fiber"
retrieve_relevant_resources(query=query, embeddings=embeddings)
top_result_and_scores(query=query, embeddings=embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get score on 1680 embeddings: 0.00059 seconds


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get score on 1680 embeddings: 0.00072 seconds
Scores: 0.6964
Text:
• Change it up a bit and experience the taste and satisfaction of other whole grains such as barley, quinoa, and bulgur. • Eat snacks high in fiber, such as almonds, pistachios, raisins, and air-popped popcorn. Add an artichoke and green peas to your dinner plate more 276 | Carbohydrates and Personal Diet Choices
Page Number: 276


Scores: 0.6810
Text:
Dietary fiber is categorized as either water-soluble or insoluble. Some examples of soluble fibers are inulin, pectin, and guar gum and they are found in peas, beans, oats, barley, and rye. Cellulose and lignin are insoluble fibers and a few dietary sources of them are whole-grain foods, flax, cauliflower, and avocados. Cellulose is the most abundant fiber in plants, making up the cell walls and providing structure. Soluble fibers are more easily accessible to bacterial enzymes in the large intestine so they can be broken down to a greater extent than

***Getting LLM for Local generation***

In [50]:
torch.tensor([1.0]).dtype

torch.float32

#### checking local gpu memory availability

In [51]:
import torch
gpu_get_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_get_memory_bytes/(2**30))
print(f"available gpu memory: {gpu_memory_gb} GB")

available gpu memory: 15 GB


In [52]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed May 22 14:00:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              28W /  70W |    601MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### we can use `gemma-2B-it` as a model

In [53]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False 
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


Loading an LLM Locally
* A quantization config- a config on what precision to load the model in (8bit, 4bit etc)
* A model ID- this will tell transformers which model/transformer to load
* A tokenizer - this turns text into numbers ready for LLM
* An LLM model - this will be what we use to genrate text based input!

In [54]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
import getpass
import os
import subprocess

# Securely prompt for the Hugging Face token
token = getpass.getpass("Enter your Hugging Face token: ")

# Set the token as an environment variable temporarily
os.environ['HUGGINGFACE_TOKEN'] = token

# Execute the login command using the environment variable
subprocess.run("huggingface-cli login --token $HUGGINGFACE_TOKEN", shell=True, check=True)

# Remove the environment variable for security
del os.environ['HUGGINGFACE_TOKEN']

Enter your Hugging Face token:  ·····································


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [56]:
# Disable tokenizers parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [57]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 801.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3


In [58]:
!pip install --upgrade bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3


In [59]:
!pip show accelerate bitsandbytes


Name: accelerate
Version: 0.30.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.10/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: catalyst
---
Name: bitsandbytes
Version: 0.43.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, torch
Required-by: 


In [60]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0))
print("Device capability:", torch.cuda.get_device_capability(0))


CUDA available: True
CUDA version: 12.1
Device name: Tesla T4
Device capability: (7, 5)


In [70]:
!pip install transformers==4.30

In [74]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 

# 1. (Optional) Setup Flash Attention 2 for faster inference
# Requires NVIDIA GPU compute capability of 8.0 or above, see: https://developer.nvidia.com/cuda-gpus
# Requires !pip install flash-attn, see: https://github.com/Dao-AILab/flash-attention 
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 2. Pick the model you want to use
model_id = "google/gemma-2b-it"  # Or any other model identifier

# 3. Instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
                                                 torch_dtype=torch.float16, # Use float16 for reduced memory usage
                                                 low_cpu_mem_usage=False, # Use full GPU memory if available
                                                 attn_implementation=attn_implementation)

# Send model to GPU if available
if torch.cuda.is_available():
  llm_model.to("cuda")

print(f"[INFO] Using model_id: {model_id}")


[INFO] Using attention implementation: sdpa


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

[INFO] Using model_id: google/gemma-2b-it


In [81]:
def get_model_num_params(model: torch.nn.Module):
  """
  This function calculates the total number of parameters in a PyTorch model.

  Args:
      model: A PyTorch model object.

  Returns:
      The total number of parameters in the model as an integer.
  """
  return sum(p.numel() for p in model.parameters())

# Get the number of parameters in llm_model
num_params = get_model_num_params(llm_model)
print(f"The model {model_id} has approximately {num_params} parameters.")


The model google/gemma-2b-it has approximately 2506172416 parameters.


In [80]:
def get_model_num_params(model: torch.nn.Module):
  """
  This function calculates the total number of parameters in a PyTorch model.

  Args:
      model: A PyTorch model object.

  Returns:
      The total number of parameters in the model as an integer.
  """
  return sum(p.numel() for p in model.parameters())

# Get the number of parameters in llm_model
num_params = get_model_num_params(llm_model)
print(f"The model {model_id} has approximately {num_params} parameters.")

def get_model_mem_size(model: torch.nn.Module):
  """
  This function calculates the estimated memory size of a PyTorch model in bytes, megabytes (MB), and gigabytes (GB).

  Args:
      model: A PyTorch model object.

  Returns:
      A dictionary containing the model memory size in bytes, MB, and GB.
  """
  mem_params = sum(param.nelement() * param.element_size() for param in model.parameters())
  mem_buffers = sum(buf.nelement() * buf.element_size() for buf in model.buffers())

  model_mem_bytes = mem_params + mem_buffers
  model_mem_mb = model_mem_bytes / (1024**2)
  model_mem_gb = model_mem_bytes / (1024**3)

  return {
      "model_mem_bytes": model_mem_bytes,
      "model_mem_mb": model_mem_mb,
      "model_mem_gb": model_mem_gb,
  }

# Get the estimated memory size of llm_model
model_mem_info = get_model_mem_size(llm_model)
print(f"The model {model_id} has an estimated memory size of:")
print(f"\t- Bytes: {model_mem_info['model_mem_bytes']}")
print(f"\t- Megabytes (MB): {model_mem_info['model_mem_mb']:.2f}")
print(f"\t- Gigabytes (GB): {model_mem_info['model_mem_gb']:.2f}")


The model google/gemma-2b-it has approximately 2506172416 parameters.
The model google/gemma-2b-it has an estimated memory size of:
	- Bytes: 5012354048
	- Megabytes (MB): 4780.15
	- Gigabytes (GB): 4.67


In [77]:
# def get_model_mem_size(model: torch.nn.Module):
#     mem_params = sum(params.nelement() * param.element_size() for param in model.parameters())
#     mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])
    
#     model_mem_bytes = mem_params + mem_buffers
#     model_mem_mb = model_mem_bytes / (1024**2)
#     model_mem_gb = model_mem_bytes / (1024**3)
    
#     return {
#         "model_mem_bytes": model_mem_bytes,
#         "model_mem_mb": model_mem_mb,
#         "model_mem_gb": model_mem_gb,
#     }

In [82]:
get_model_mem_size(llm_model)

{'model_mem_bytes': 5012354048,
 'model_mem_mb': 4780.1533203125,
 'model_mem_gb': 4.668118476867676}

#### Generating text with our LLM

In [86]:
input_text = "Query"
print(f"Input Text: \n{input_text}")

# Create a prompt template for instruction tune model
dialogued_template = [
    {"role":"user",
    "content":input_text}
]
#apply
prompt = tokenizer.apply_chat_template(conversation=dialogued_template,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input Text: 
What are the macronutrients, and what roles do they play in the human body?

Prompt (formatted):
<bos><start_of_turn>user
What are the macronutrients, and what roles do they play in the human body?<end_of_turn>
<start_of_turn>model



In [87]:
%%time
#tokenize it(make a number of sentences)  and send it to gpu
input_ids = tokenizer(prompt,
                     return_tensors="pt").to("cuda")
# generate outputs from local llm
# so token in token out
outputs = llm_model.generate(**input_ids,
                            max_new_tokens=256)
print(f"Model output (tokens): \n{outputs[0]}\n")

Model output (tokens): 
tensor([     2,      2,    106,   1645,    108,   1841,    708,    573, 186809,
        184592, 235269,    578,   1212,  16065,    749,    984,   1554,    575,
           573,   3515,   2971, 235336,    107,    108,    106,   2516,    108,
         21404, 235269,   1517, 235303, 235256,    476,  25497,    576,    573,
        186809, 184592,    578,   1024,  16065,    575,    573,   3515,   2971,
        235292,    109,    688,  12298,   1695, 184592,  66058,    109, 235287,
          5231, 156615,  56227,  66058,    108,    141, 235287,  34428,   4134,
           604,    573,   2971, 235303, 235256,   5999,    578,  29703, 235265,
           108,    141, 235287, 110165,  56227,    708,    573,   7920,   4303,
           576,   4134,    604,   1546,   5999, 235265,    108,    141, 235287,
         25280,  72780,    708,   1941,    674,   1987,   5543,    577,  55997,
        235269,   1582,    685,   3733,  29907, 235269,  16803, 235269,    578,
         19574, 

In [ ]:
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (deocded):\n{outputs_decoded}\n")

: 

In [ ]:
# print(f"Input text: {input_text}\n")
# print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")

: 

In [ ]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "Some question ask to gpt"
]

# Manually created question list
manual_questions = [
   "manual questions"
]

query_list = gpt4_questions + manual_questions
query_list

: 

In [112]:
import random
query = random.choice(query_list)

print(f"Query: {query}")

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

Query: How often should infants be breastfed?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get score on 1680 embeddings: 0.00013 seconds


(tensor([0.6205, 0.6067, 0.5696, 0.5624, 0.5307], device='cuda:0'),
 tensor([1151, 1160, 1144, 1138, 1155], device='cuda:0'))

### Augmenting our prompt with context items


In [ ]:
# pages_and_chunks[420]

: 

In [115]:
def prompt_formatter(query: str,
                    context_items: list[dict]) -> str:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items]) 
    base_prompt = """Based on the following context items, please answer the following query in a comprehensive and informative way.

**Reading Comprehension:**

* Before responding, carefully analyze the context to identify the most relevant passages that address the query.
* Focus on extracting key information and relationships within the context.

**Answer Formatting:**

* Do not include your thought process or reasoning in the answer.
* Present the answer in a clear, concise, and well-structured manner.
* Use complete sentences and proper grammar.

**Example Answer Style:**

Refer to the following examples for the ideal answer format:

**Example 1:**

Context: A press release announces the launch of a new electric car with a driving range of 300 miles.

Query: What is the driving range of the new electric car?

Answer: The new electric car boasts a driving range of 300 miles on a single charge.

**Example 2:**

Context: A historical document describes the events leading up to the signing of a peace treaty.

Query: What were the key factors that led to the signing of the peace treaty?

Answer: The signing of the peace treaty was driven by several key factors, including mounting war fatigue on both sides, significant casualties, and the emergence of a new leader who advocated for peace.

**Now apply these principles to answer the following query based on the provided context:**

{context}

**Relevant Passages:**

<Identify and extract relevant passages from the context here>

**User Query:**

{query}

**Answer:**

[Write your answer here. Focus on providing a clear and informative response based on the extracted relevant passages and the context.]

    """
    base_prompt = base_prompt.format(context=context, query=query) 
    
    # Create prmpt template for instruction tune model
    dialogue_template = [
        {
            "role":"user",
            "content":base_prompt
        }
    ]
    
    #apply the template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    
    
    return prompt

query = random.choice(query_list)
print(f"Query: {query}")

# get relevant resources
scores, indices = retrieve_relevant_resources(query=query,
                                             embeddings=embeddings)

# Create a list of context items
context_items = [pages_and_chunks[i] for i in indices]

# Format our prompt
prompt = prompt_formatter(query=query, context_items=context_items)

print(prompt)


Query: Describe the process of digestion and absorption of nutrients in the human body.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get score on 1680 embeddings: 0.00059 seconds
<bos><start_of_turn>user
Based on the following context items, please answer the following query in a comprehensive and informative way.

**Reading Comprehension:**

* Before responding, carefully analyze the context to identify the most relevant passages that address the query.
* Focus on extracting key information and relationships within the context.

**Answer Formatting:**

* Do not include your thought process or reasoning in the answer.
* Present the answer in a clear, concise, and well-structured manner.
* Use complete sentences and proper grammar.

**Example Answer Style:**

Refer to the following examples for the ideal answer format:

**Example 1:**

Context: A press release announces the launch of a new electric car with a driving range of 300 miles.

Query: What is the driving range of the new electric car?

Answer: The new electric car boasts a driving range of 300 miles on a single charge.

**Example 2:**



In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

#Generate an output of tokens
outputs = llm_model.generate(**input_ids, temperature = 0.7, #temeperature less measn more deterministic and more temp means creative/randomness
                            do_sample=True, # whether or not to use sampling
                            max_new_tokens=256)

#turn the output into text
output_text = tokenizer.decode(outputs[0])
print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt,'')}")

: 

# Functionize our LLM answer feature

ask("What kind of nutrients we have?")

In [119]:
def ask(query: str, 
       temperature: float=0.7,
       max_new_tokens: int=256,
       format_answer_text=True,
       return_answer_only=True):
    """
    Takes a query, find relevant resources and generates an answer to the query based on the relevant resources
    """
    scores, indices = retrieve_relevant_resources(query=query,
                                                 embeddings=embeddings)
    
    # Create a list of context items
    context_items = [pages_and_chunks[i] for i in indices]
    
    #add scores to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu()
        
    # AUGMENTAION
    prompt = prompt_formatter(query=query,
                             context_items=context_items)
    
    # GENERATIONS
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                temperature=temperature,
                                do_sample=True,
                                max_new_tokens=max_new_tokens)
    
    # decode to texts
    output_text = tokenizer.decode(outputs[0])
    
    
    # Format the answer
    if format_answer_text:
        output_text = output_text.replace(prompt,"").replace("<bos>","").replace("<eos>","")
        
    # Only return answer without context items
    if return_answer_only:
        return output_text
    
    return output_text, context_items

In [124]:
query = "You query"
print(f"Query: {query}")
ask(query=query,
   temperature=0.2,
   return_answer_only=True)

Query: Tell me something about B12


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get score on 1680 embeddings: 0.00072 seconds


"Sure, here's the answer to the user's query:\n\nThe context provides information about the dietary sources of vitamin B12 and its importance for human health. It emphasizes that vitamin B12 is found naturally in animal products and fortified breakfast cereals, but it is not present in plant foods. The context also discusses the importance of proper absorption of vitamin B12, which depends on the functioning of the stomach, pancreas, and small intestine.\n\nThe passage also highlights the connection between vitamin B12 and folate, emphasizing that a deficiency in vitamin B12 can lead to similar health consequences as folate deficiency. It also mentions that vitamin B12 deficiency can cause macrocytic anemia in children and adults, and that pregnant women born to cobalamin-deficient mothers are at an increased risk for neural-related health problems."